<a href="https://colab.research.google.com/github/zahidhamidi/ML-Project/blob/main/Unit_test_RBM_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from tqdm.auto import tqdm
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_excel("/content/sample_data/robust_test_dataset.xlsx")

In [6]:
def text_extraction(text,topics):

  # Define the topics as flexible terms (subwords or whole words)
  # topics = ["gas shows" , "oil show", "overpressure", "fluid communication" , "gas bearing" , "h2s" , "co2" , "stuck pipe" , "pressure build up", "sand production"]


  # Replace hyphens with blank spaces in all items in the topics list
  topics = [topic.replace('-', ' ') for topic in topics]


  # Initialize a list to store selected sentences
  selected_sentences = []

  # Initialize a flag to check for both topic terms and negation terms
  both_detected = False

  # Tokenize the text into sentences using NLTK's sent_tokenize
  sentences = sent_tokenize(text)

  # Initialize flags to check for both topic terms and negation terms
  topic_found = False
  negation_detected = False

  if len(sentences) == 1:
    selected_sentences.append(text)
    # print(0)

  else:

    for sentence in sentences:

      # remove apostrophes from string
      sentence = sentence.replace("'", "")
      sentence = sentence.replace(",", "")

      # Tokenize the sentence into words
      lemmatized_words = word_tokenize(sentence)

      # Join the lemmatized words back into a sentence
      lemmatized_sentence = ' '.join(lemmatized_words)

      # Initialize an empty pattern string
      pattern_string = r'(?i)(?:(?<=\s)|^)'

      for topic_pattern in topics:
        words = topic_pattern.split()

        if len(words) >= 2:

          # Split the topic words by spaces and hyphens and create a pattern to match them
          # topic_pattern = re.compile(r'(?i)\b' + r'\b|\b'.join(re.escape(topic) for topic in topics) + r'\b')

          # Create a regex pattern to match the multi-word topic with any characters in between
          pattern_string = r'(?i)\b' + r'\s*[,.\s]*\s*'.join(re.escape(word) for word in words) + r'\b'

          # Compile the regex pattern
          topic_pattern = re.compile(pattern_string, re.IGNORECASE)


          if re.search(topic_pattern, lemmatized_sentence.lower()):
                topic_found = True
                break

        elif len(words) != 0:
            single_pattern = rf'(?i)(?:(?<=\s)|^)({re.escape(words[0])})(?=\s|$)'
            if re.search(single_pattern, lemmatized_sentence.lower()):
                topic_found = True
                break

      # Check if the lemmatized sentence contains negation terms
      if re.search(negation_terms, lemmatized_sentence.lower()):
          negation_detected = True
      else:
          negation_detected = False

      counter = 0
      # If a topic term is found and either no negation terms are found or negation_detected is False, append the sentence
      if (topic_found is True) and (negation_detected is True):
          selected_sentences.append(lemmatized_sentence)
          both_detected = True
          # print(1)
          break

      elif (topic_found is True) and (negation_detected is False):
          selected_sentences.append(lemmatized_sentence)
          both_detected = False
          counter = counter + 1
          # print(2)

          if counter >1:
            break
          break

      # else:
      #   selected_sentences.append(sentences)



    # If no sentence with both topic and negation terms is found, append the first sentence without a negation term
    if topic_found is False:
        selected_sentences.append(text)

  # print("Selected Sentence: ", selected_sentences)
  # print("Topic word found: ",topic_found)
  # print("Negation word found: ",negation_detected)

  return selected_sentences








In [4]:
def unit_test(text,topic):
  selected = text_extraction(text,topic)
  print(selected)

In [9]:
finalized_text = []

topic = "gas show"

for doc in tqdm(df["doc_text_original"]):
  selected = text_extraction(doc,topic)

  if len(selected) == 1:
    finalized_text.append(selected[0])

  else:
    sentence = ".".join(selected)
    finalized_text.append(sentence)




  # Add the selected_sentences as a new column named 'selected_sentence'
  # df['selected_sentence'] = selected

 # Add the selected_sentences as a new column named 'selected_sentence'
df['selected_sentence'] = finalized_text


  0%|          | 0/2161 [00:00<?, ?it/s]

In [14]:
# Define the negation terms pattern with case-insensitivity
negation_terms = r'(?i)(?:(?<=\s)|^)(no|non|unlikely|none|not|nor|without|lack|rather)(?=\s|$|\b)'


# Determine the prediction (0 or 1) based on the presence of negation terms in selected_sentence
for index, row in df.iterrows():
    selected_sentence = row['selected_sentence']

    if selected_sentence is not None:
        # Check if the lemmatized sentence contains negation terms
        if re.search(negation_terms, selected_sentence.lower()):
            prediction = 0  # Negation terms found, set the prediction to 0
        else:
            prediction = 1  # No negation terms found, set the prediction to 1
    else:
        prediction = 0  # Handle the case where selected_sentence is None

    df.at[index, 'predicted'] = prediction

# Print the DataFrame with the 'label' column
df[['doc_text_original', 'selected_sentence', 'label_code', 'predicted']]

,doc_text_original,selected_sentence,label_code,predicted
0,"'Shows: (0il, Gas, â€˜ Show of gas while drill...","'Shows: (0il, Gas, â€˜ Show of gas while drill...",1,1.0
1,"""VI. Reservoirs: Sand reservoirs are present i...",A thin 8 foot sand in the lower portipn of the...,1,1.0
2,"'Shows: (0il, Gas, â€˜ Show of gas while drill...","'Shows: (0il, Gas, â€˜ Show of gas while drill...",1,1.0
3,"""Drilling shale as above with _ occasional. st...","""Drilling shale as above with _ occasional. st...",1,1.0
4,'74 | -~ | 73| Drilling shale as above with â€...,'74 | -~ | 73| Drilling shale as above with â€...,1,1.0
...,...,...,...,...
2156,21 21A 22 22A 23 23A 24 24A 25 25A 26 26A 27 2...,21 21A 22 22A 23 23A 24 24A 25 25A 26 26A 27 2...,1,0.0
2157,"""used to calculate the geochemical Altbough th...",The 1761.8m & till has quite a low value ident...,1,1.0
2158,"""used to calculate the geochemical Altbough th...",The 1761.8m & till has quite a low value ident...,1,1.0
2159,lower levels of these unknown compounds is not...,sufficiently understood as yet it is not possi...,1,0.0


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

accuracy_values = []
precision_values = []
recall_values = []
f1_values = []

In [16]:
# Assuming you have loaded your DataFrame 'df' with the columns 'label_code' and 'predicted_label_code'
y_true = df['label_code']
# y_pred = df['prediction_negex']
y_pred = df['predicted']

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Calculate additional evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Append metric values to the respective lists
accuracy_values.append(accuracy)
precision_values.append(precision)
recall_values.append(recall)
f1_values.append(f1)

print(accuracy, precision, recall, f1)

0.8056455344747802 0.6698925524299644 0.7274762085166873 0.6885628638957744


In [17]:
# Assuming 'df' is your DataFrame
df.to_excel("dataset_gb.xlsx", index=False)